## HW8 Exploring Unfairness in Data ⚖️

## 1. Unfairness

It is natural to assume that a model built from "real-world" data will inherently represent the world-at-large. We often take the data that we have for granted, especially when we are first getting started with Data Science. However, if we do not pay attention to what our data look like, how they were collected, and what features they contain, we may unknowingly create models that propagate cultural biases and unfairnesses.


In 2014, Amazon began building programs that could automate the hiring process for engineers. They wanted a machine to be able to pick out the top resumes from the thousands they receive every year. They trained their model on all of the resumes that they had, hoping that the model would be able to identify trends in keyword frequency within those applications. If most applications contained the word "intern," then one might reasonably expect that a resume containing it would be ranked higher than one that doesn't. However, as they began to deploy their model, it became increasingly apparent that the model was discriminating against women. When engineers investigated why this was the case, they found that the data they trained the model with, the resumes, had mostly come from men. The model had learned to prefer resumes that didn't contain the word "women's" because that word wasn't frequent seen during its training. Although gender was not explicitly a feature of the dataset, it was still present in the dataset, encoded within the experiences that applicants reported. 

Amazon's case serves as a reminder that we must be careful of our data, even more so today as data becomes cheaper to collect.

## 2. Exploring Loan Approval Data

Imagine that you are a data scientist at a bank and that one of your company's primary business areas is in lending money. The current loan approval process, that has been in place since the founding of the bank, has always relied on manual review of applications -- a process that is tedious and doesn't scale well in the modern age. The company wants to expand their business, but this archeic system is holding them back.

Think about how to approach this problem, you immediately think of using the bank's past loan approval records to build a model that can learn how a human application reviewer decides which applications to approve and which to reject.


### Acquiring the Data



In [ ]:
from os.path import exists


assert exists(f'loan-payments.csv'), 'Loan data file is missing.'

Next, let's load our data. In the cell below, we read our [CSV][1] file into a [Pandas][2] [`DataFrame`][3] called `data`.



[1]: https://en.wikipedia.org/wiki/Comma-separated_values
[2]: https://pandas.pydata.org/
[3]: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html

In [ ]:
import pandas as pd

data = pd.read_csv(f'loan-payments.csv')

Let's take a look at what we have.

In [ ]:
data

**Q1.a** How many examples are in our data set? How many features does it have?

**Q1.b** Come up with a description of what you think each feature is and what type of feature each one is. Which one should be our target variable? Which ones do you think will be useful for our model?

### Making Some Adjustments

Now let's drop the columns in `data` that contain features that we are not interested in. Since `loan_id`s are not informative for predicting new loans, we can ignore them. Additionally, `effective_date`, `due_date`, and `paid_time_off` are all encoded in `past_due_days`. It is unlikely that the specifics of when a loan was due is predictive of success.

In [ ]:
not_interested = ['loan_id', 'effective_date', 'due_date', 'paid_off_time']

data = data.drop(not_interested, axis=1)

Let's see our new data set.

In [ ]:
data.head()

Did you notice that `past_due_days` has `NaN` values?

**Q2.a** Why might some of the values in `past_due_days` be `NaN`?  Discuss what we should do about these values and note your conclusion below.

**Try this!** Replace the values in `past_due_days` with a reasonable value. `HINT` you can use the `fillna` function on `DataFrame`s to do this.

In [ ]:
# Replace NaN values in 'past_due_days' with 0, assuming that NaN means not past due
data['past_due_days'] = data['past_due_days'].fillna(0)


Let's see if it worked.

In [ ]:
data.head()

Nice!

### Visualizing the Data Set

Now that we have narrowed down the features we want to use, let's visualize them.

**Try this!** For each feature, show the plot that we can use to understand the values of that feature. These plots should be appropriate for the type of each feature (e.g. use a bar plot for categorical features). Ensure that you have all the components off a nice plot, making sure to include things like axes labels, a legend, and a title. Also include a `raw` cell below each, describing what you see.

In [ ]:
import matplotlib.pyplot as plt

# this cell is free!

# your code here
# Plot for 'principal' (continuous feature)
plt.figure(figsize=(6,4))
plt.hist(data['principal'], bins=20, color='skyblue', edgecolor='black')
plt.title('Distribution of Principal')
plt.xlabel('Principal')
plt.ylabel('Number of Loans')
plt.grid(True)
plt.show()


# Plot for 'terms' (discrete numeric feature)
plt.figure(figsize=(6,4))
data['terms'].value_counts().sort_index().plot(kind='bar', color='salmon', edgecolor='black')
plt.title('Distribution of Terms')
plt.xlabel('Terms (days)')
plt.ylabel('Number of Loans')
plt.grid(axis='y')
plt.show()

# Plot for 'past_due_days' (may be highly skewed)
plt.figure(figsize=(6,4))
plt.hist(data['past_due_days'], bins=20, color='seagreen', edgecolor='black')
plt.title('Distribution of Past Due Days')
plt.xlabel('Past Due Days')
plt.ylabel('Number of Loans')
plt.grid(True)
plt.show()

# Plot for 'age' (continuous feature)
plt.figure(figsize=(6,4))
plt.hist(data['age'], bins=15, color='orange', edgecolor='black')
plt.title('Distribution of Age')
plt.xlabel('Age')
plt.ylabel('Number of Loans')
plt.grid(True)
plt.show()

# Plot for 'education' (categorical)
plt.figure(figsize=(6,4))
data['education'].value_counts().plot(kind='bar', color='mediumpurple', edgecolor='black')
plt.title('Distribution of Education Levels')
plt.xlabel('Education')
plt.ylabel('Number of Loans')
plt.grid(axis='y')
plt.show()


**Q2.b** Describe what you see in each of the plots above.

## 3. Building a Model

Now that we have a sense for the nuances of our dataset we can try building some models.



Before we continue, we will need to encode our categorical features with enumerations instead of the string values that they currently have. As a reminder, this is what our dataset looks like right now.

In [ ]:
data.head()

An easy way to do this encoding is to use the [LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) from `sklearn`. In the cell below, we create a list called `categorical` containing the names of the columns corresponding to the categorical features in our dataset. We then create and instance of a `LabelEncoder` and use it to transform the categorical features.

In [ ]:
from sklearn.preprocessing import LabelEncoder

categorical = ['loan_status', 'education', 'gender']

# create an instance of a LabelEncoder
encoder = LabelEncoder()

# make a copy of our data
encoded = data.copy()

# apply the encoder's `fit_transform` method to the values for each categorical
# feature column
encoded[categorical] = data[categorical].apply(encoder.fit_transform)

Let's take a look at the results.

In [ ]:
encoded

Notice how the categorical values like "PAIDOFF" have now been replaced with numbers. We can see which numbers map to each value like this:

In [ ]:
for column in categorical:
    print(*sorted(zip(encoded[column].unique(), data[column].unique()), key=lambda x: x[0]))

Next let's separate our features from our target variable, `loan_status`.

In [ ]:
X, y = encoded.loc[:, encoded.columns != 'loan_status'], encoded.loan_status

### Establishing a Baseline

Now we're ready to start building models. First, let's create a train/test split of our data.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=3)

Then, let's train and evaluate a LogisticRegression model.

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='liblinear', multi_class='auto')
model.fit(X_train, y_train)

**Try this!** In the cell below, evaluate the model's performance on the testing set.

In [ ]:
# your code here
from sklearn.metrics import accuracy_score, classification_report

# Make predictions on the test set
y_pred = model.predict(X_test)

# Print the accuracy
print(f"Test set accuracy: {accuracy_score(y_test, y_pred):.3f}")

# Print a detailed classification report
print("\nClassification Report:\n", classification_report(y_test, y_pred))


**Q3.a** How does our model perform on the test set?

Let's also try looking at the model's performance on test examples of different genders.

In [ ]:
print(f'''
validation (men) score: {model.score(X_test[X_test['gender'] == 1], y_test[X_test['gender'] == 1]):0.3f}
validation (women) score: {model.score(X_test[X_test['gender'] == 0], y_test[X_test['gender'] == 0]):0.3f}
''')

Yikes!

**Q3.b** What do you notice about these scores? How does these compare with the initial score we saw for the entire test set? What does this imply about our model?

### Dropping Gender

So our model is biased with respect to gender and gender is a feature of the model. Would it help to ignore the gender feature during training? Let's try it out.

Let's start by creating another train/test split, but this time using a copy of `X` and `y` that don't include `gender`.

In [ ]:
X_without_gender = X.drop(['gender'], axis=1)

X_train, X_test, y_train, y_test = \
    train_test_split(X_without_gender, y, test_size=0.2, stratify=y, random_state=3)

Let's see what `X_train` looks like now.

In [ ]:
X_train.head()

Now let's repeat our procedure for our baseline experiment.

In [ ]:
model = LogisticRegression(solver='liblinear', multi_class='auto')
model.fit(X_train, y_train)

print(f'''

validation score: {model.score(X_test, y_test)}
validation (men) score: {model.score(X_test[X.iloc[X_test.index]['gender'] == 1],
                                     y_test[X.iloc[X_test.index]['gender'] == 1]):0.3f}
validation (women) score: {model.score(X_test[X.iloc[X_test.index]['gender'] == 0],
                                       y_test[X.iloc[X_test.index]['gender'] == 0]):0.3f}
''')

**Q4.a** Did the results change?

**Q4.b** Discuss what this might imply about our model and our data. 

**Q4.c** Also, discuss why it may or may not be a good idea to ignore "protected variables" like "gender" when training a model. Record your response below.
